In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
!pip install pytorch-lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 796 kB 4.8 MB/s 
     |████████████████████████████████| 136 kB 86.0 MB/s 
     |████████████████████████████████| 529 kB 66.5 MB/s 
     |████████████████████████████████| 87 kB 8.2 MB/s 
  Created wheel for fire: filename=fire-0.4.0-py2.py3-none-any.whl size=115942 sha256=e695d244bb74864e3b298939663ce5c5d90c2eeb188fbbe9cd0c0c4f27ecb79c
  Stored in directory: /root/.cache/pip/wheels/8a/67/fb/2e8a12fa16661b9d5af1f654bd199366799740a85c64981226
Successfully built fire


In [3]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import lr_scheduler, Adam
from torch.functional import F
from torch.nn.utils.rnn import pad_sequence

import pytorch_lightning as pl
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger

import pandas as pd
import string
from matplotlib import pyplot as plt

In [4]:
batch_size = 256
n_epochs = 200
hidden_size = 128
embedding_dims = 16
n_layers = 1
keep_prob = 0.2

In [31]:
data_path = "/content/drive/MyDrive/Datasets/LSTM/Dinosaur Name Generator/dinosaur.txt"
with open(data_path, "r") as fp:
  names = fp.read().lower().splitlines()

EOS = "EOS"

split_names = []
for name in names:
  split_name = list(name) + [EOS]
  split_names.append(split_name)

vocab = [EOS] + list(string.ascii_lowercase)
idx2char = dict(enumerate(vocab))
char2idx = {char: idx for idx, char in enumerate(vocab)}

encoded_names = []
for name in split_names:
  encoded_name = torch.tensor([char2idx[char] for char in name])
  encoded_names.append(encoded_name)

padded_names = pad_sequence(encoded_names, batch_first=True, padding_value=0)

In [11]:
class DinoDataset(Dataset):
  def __init__(self, samples):
    self.samples = samples

  def __len__(self):
    return len(self.samples)

  def __getitem__(self, idx):
    sample = self.samples[idx]
    X = sample[:-1]
    y = sample[1:]
    return X, y

In [47]:
class DinoDatamodule(pl.LightningDataModule):
  def __init__(self, samples):
    super().__init__()
    self.samples = samples

  def setup(self, stage=None):
    self.train_set = DinoDataset(self.samples)

  def train_dataloader(self):
    return DataLoader(self.train_set, batch_size=batch_size, shuffle=True)

In [51]:
class LSTM(pl.LightningModule):
  lr = 0.001
  
  def __init__(self, vocab_size, embedding_dims, hidden_size, n_layers, keep_prob):
    super().__init__()

    self.vocab_size = vocab_size
    self.embedding_dims = embedding_dims
    self.hidden_size = hidden_size
    self.n_layers = n_layers
    self.keep_prob = keep_prob

    self.criterion = nn.CrossEntropyLoss()

    self.embedding = nn.Embedding(num_embeddings=self.vocab_size, embedding_dim=self.embedding_dims)
    self.lstm = nn.LSTM(input_size=self.embedding_dims, hidden_size=self.hidden_size, num_layers=self.n_layers, batch_first=True)
    self.dropout = nn.Dropout(p=self.keep_prob)
    self.fc = nn.Linear(in_features=self.hidden_size, out_features=self.vocab_size)

  def forward(self, X, state_previous):
    h_previous, c_previous = state_previous
    embedding = self.embedding(X)
    y_hat, state_current = self.lstm(embedding, (h_previous, c_previous))
    y_hat = self.dropout(y_hat)
    y_hat = self.fc(y_hat)

    return y_hat, state_current

  def training_step(self, batch, batch_idx):
    X, y = batch
    h_previous = torch.zeros(n_layers, X.size(0), hidden_size).to(self.device) 
    c_previous = torch.zeros(n_layers, X.size(0), hidden_size).to(self.device)

    y_hat, state_current = self.forward(X, (h_previous, c_previous)) 

    loss = self.criterion(y_hat.transpose(1, 2), y)

    return loss

  def configure_optimizers(self):
    return Adam(self.parameters(), self.lr)
    

In [52]:
dm = DinoDatamodule(padded_names)

In [53]:
lstm_model = LSTM(
    vocab_size=len(char2idx),
    embedding_dims=embedding_dims,
    hidden_size=hidden_size, 
    n_layers=n_layers, 
    keep_prob=keep_prob
    )

trainer = Trainer(
    max_epochs=200,
    logger=None,
    gpus=1,
    )

trainer.fit(lstm_model, dm) 

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:447: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  f"Setting `Trainer(gpus={gpus!r})` is deprecated in v1.7 and will be removed"
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name      | Type             | Params
-----------------------------------------------
0 | criterion | CrossEntropyLoss | 0     
1 | embedding | Embedding        | 432   
2 | lstm      | LST

Training: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=200` reached.


In [54]:
torch.save(lstm_model.state_dict(), "dino_name_generator_v2.pt")

In [55]:
lstm_model = LSTM(
    vocab_size=len(char2idx),
    embedding_dims=embedding_dims,
    hidden_size=hidden_size, 
    n_layers=n_layers, 
    keep_prob=keep_prob
    )

lstm_model.load_state_dict(torch.load("/content/drive/MyDrive/Datasets/LSTM/Dinosaur Name Generator/dino_name_generator_v2.pt"))

<All keys matched successfully>

In [56]:
# model.eval()
characters = []
character_idx = torch.randint(low=1, high=len(vocab), size=(1,1))
characters.append(character_idx.squeeze().item())
h_previous = torch.zeros(n_layers, character_idx.size(0), hidden_size)
c_previous = torch.zeros(n_layers, character_idx.size(0), hidden_size)

with torch.no_grad():
  while character_idx.squeeze().item() != char2idx["EOS"]:
    y_hat, state_current = lstm_model(character_idx, (h_previous, c_previous))
    softmax_y_hat = F.softmax(y_hat, dim=-1)
    character_idx = torch.argmax(softmax_y_hat, dim=-1)
    characters.append(character_idx.squeeze().item())

    h_previous, c_previous = state_current 

name = "".join([idx2char[character] for character in characters if character != 0])
authenticity = "real dinosaur" if name in names else "made up"
print(f"THE MODEL HAVE GENERATED THE NAME \"{name}\", WHICH IS A **{authenticity}** NAME")
if name not in names:
  with open("/content/drive/MyDrive/Datasets/LSTM/Dinosaur Name Generator/gen_names.txt", "a+") as fp:
    fp.seek(0)
    fp_names = fp.read().splitlines()
    if name not in fp_names:
      fp.seek(0, 2)
      fp.write(f"{name}\n") 

THE MODEL HAVE GENERATED THE NAME "ganthosaurus", WHICH IS A **made up** NAME
